# Spatial Relationships

Lorum Ipsum

In [ ]:
import sys, os
import urllib.parse
import requests
import rasterio
import json

import matplotlib.pyplot as plt
import pandas as pd 
import geopandas as gpd
import numpy as np

from io import BytesIO
from rasterio.plot import show
from rasterstats import zonal_stats
from contextlib import contextmanager
from rasterio.io import MemoryFile
from rasterio.mask import mask
from shapely import box

In [ ]:
# Choose and Map your favourite country
iso3 = "GHA"

# Define some input variables for data exploration
### Official World Bank Group boundary dataset - https://datacatalog.worldbank.org/search/dataset/0038272/World-Bank-Official-Boundaries
# We can also read these from the ESRI rest endpoint through the World Bank's GeoWB
queryable_geowb_url_adm2 = urllib.parse.quote(f'https://services.arcgis.com/iQ1dY19aHwbSDYIF/arcgis/rest/services/World_Bank_Global_Administrative_Divisions/FeatureServer/3/query?where="ISO_A3"=\'{iso3}\'&outFields=*&f=pgeojson', safe=':/?=&')
queryable_geowb_url_adm1 = urllib.parse.quote(f'https://services.arcgis.com/iQ1dY19aHwbSDYIF/arcgis/rest/services/World_Bank_Global_Administrative_Divisions/FeatureServer/2/query?where="ISO_A3"=\'{iso3}\'&outFields=*&f=pgeojson', safe=':/?=&')
#country_bound = gpd.read_file(queryable_geowb_url)

# SSL errors are common when working inside the World Bank firewall. If you encounter these, you can bypass them with the code below
response = requests.get(queryable_geowb_url_adm2, verify=False) # <--- Ignore SSL
adm2_bounds = gpd.read_file(BytesIO(response.content))

response = requests.get(queryable_geowb_url_adm1, verify=False) # <--- Ignore SSL
adm1_bounds = gpd.read_file(BytesIO(response.content))

## Compare intersects and within

Identifying intersecting and containing geographies are the most common spatial relationships, and they are very similar

In [ ]:
# Optional - explore the data in an interactive map
# adm1_bounds.explore().save('adm1_bounds.html')
# select a single administrative 1 and select intersecting administrative 2 boundaries
sel_adm1 = adm1_bounds.loc[adm1_bounds['ADM1CD_c'] == 'GHA003']
sel_adm2_intersect = adm2_bounds.loc[adm2_bounds.intersects(sel_adm1.geometry.squeeze())]

sel_adm2 = adm2_bounds.loc[adm2_bounds.within(sel_adm1.geometry.squeeze())]
sel_adm2_within = adm2_bounds.loc[adm2_bounds.within(sel_adm1.geometry.squeeze())]

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

sel_adm1.plot(ax=axes[0])
axes[0].set_title('Selected ADM1')
sel_adm2_intersect.plot(ax=axes[1])
axes[1].set_title('Intersecting ADM2')
sel_adm2_within.plot(ax=axes[2])
axes[2].set_title('Within ADM2')

## Extract raster data

In [ ]:
### These functions come from a GOST library of miscellaneous geospatial functions called GOSTrocks 
# that I have developed over the years. You can find the code for these functions at the link
# below, but I have copied them here as exploring the code is useful for learning.
# https://github.com/worldbank/gostrocks
@contextmanager
def create_rasterio_inmemory(src, curData):
    """Create a rasterio object in memory from a numpy array

    Parameters
    ----------
    src : rasterio.metadata
        Metadata for the rasterio object
    curData : numpy.ndarray
        Data to write to the rasterio object

    Yields
    ------
    rasterio.DatasetReader
        In-memory rasterio dataset
    """
    with MemoryFile() as memFile:
        with memFile.open(**src) as dataset:
            try:
                dataset.write(curData)
            except Exception:
                dataset.write_band(1, curData)
            del curData

        with memFile.open() as dataset:
            yield dataset

def clipRaster(inR, inD, outFile=None, crop=True):
    """Clip input raster to provided geodataframe

    Parameters
    ----------
    inR : rasterio.DatasetReader
        Input raster to clip
    inD : geopandas.GeoDataFrame
        GeoDataFrame containing the clipping geometry
    outFile : str, optional
        File to write the clipped raster to, by default None
    crop : bool, optional
        Whether to crop the raster to the unary_union of the GeoDataFrame (True) or to the bounding box (False), by default True

    Returns
    -------
    tuple
        A tuple containing the clipped raster and its metadata.
    """
    if isinstance(inR, str):
        inR = rasterio.open(inR)
    if isinstance(inD, str):
        inD = gpd.read_file(inD)
    if inD.crs != inR.crs:
        inD = inD.to_crs(inR.crs)
        inD = inD.buffer(0)
    out_meta = inR.meta.copy()

    def getFeatures(gdf):
        # Function to parse features from GeoDataFrame in such a manner that rasterio wants them
        return [json.loads(gdf.to_json())["features"][0]["geometry"]]

    if crop:
        tD = gpd.GeoDataFrame([[1]], geometry=[inD.union_all()])
    else:
        tD = gpd.GeoDataFrame([[1]], geometry=[box(*inD.total_bounds)])

    coords = getFeatures(tD)
    out_img, out_transform = mask(inR, shapes=coords, crop=True, all_touched=True)
    out_meta.update(
        {
            "driver": "GTiff",
            "height": out_img.shape[1],
            "width": out_img.shape[2],
            "transform": out_transform,
        }
    )
    if outFile:
        with rasterio.open(outFile, "w", **out_meta) as dest:
            dest.write(out_img)
    return [out_img, out_meta]

In [ ]:
# Define our raster data - this is a Global Map of Landcover from 2015, produced by the European Space Agency (ESA) 
# - https://www.esa-landcover-cci.org/?q=node/175
esa_landcover_ddh = "https://datacatalogfiles.worldbank.org/ddh-published/0066950/DR0095711/ESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7.tif"

with rasterio.Env(GDAL_HTTP_UNSAFESSL='YES'): #bypass SSL errors from World Bank firewall
    landcover_r = rasterio.open(esa_landcover_ddh)
    lc_data, lc_meta = clipRaster(landcover_r, sel_adm1) #this function returns a numpy array and metadata dictionary that we can use to create a rasterio object in memory
    with create_rasterio_inmemory(lc_meta, lc_data) as lc_r:
        show(lc_r)

In [ ]:
with rasterio.Env(GDAL_HTTP_UNSAFESSL='YES'):
    landcover_r = rasterio.open(esa_landcover_ddh)
    lc_data, lc_meta = clipRaster(landcover_r, sel_adm1, crop=False) #the crop parameter determines whether to crop to the bounding box of the geometry (False) or to the unary_union of the geometry (True)
    with create_rasterio_inmemory(lc_meta, lc_data) as lc_r:
        show(lc_r)

In [ ]:
lc_data.shape

In [ ]:
## We can also use rasterstats to calculate zonal statistics for the intersecting and within administrative 2 boundaries, 
# using the clipped raster data. This is a common workflow in geospatial data science, 
# where we want to summarize raster data (e.g. landcover) within vector boundaries (e.g. administrative boundaries).
lc_counts = zonal_stats(sel_adm2_intersect, esa_landcover_ddh, categorical=True)
lc_counts = pd.DataFrame(lc_counts, index=sel_adm2_intersect['ADM2CD_c']).sort_index(axis=1)
lc_counts

## Challenge Question

Using the results of the zonal stats, create a 3-panel map comparing the percent coverage of three landcover types of your choice. Make sure to include the proper names of the Landcover classes, not just their numbers.